In [1]:
!pip install tensorflow_text
!pip install nlpaug
#!pip install tensorflow-addons
#!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 45.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fo

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import nlpaug.augmenter.word as naw
import re
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
tf.keras.mixed_precision.set_global_policy('mixed_float16')
import xgboost as xgb


In [3]:
ds = pd.read_csv('/content/train_output - train_output(1).csv')

In [4]:
def preprocess_text(text):
    text = re.sub(r'\n', ' ', text)
    return text

ds['Text'] = ds['Text'].apply(preprocess_text)

In [5]:
ds.dropna(axis=0, how='any')

,Text,Label
0,"IN THE HIGH COURT OF KARNATAKA, ...",PREAMBLE
1,BEFORE THE HON'BLE MR.JUSTICE ANAND B...,PREAMBLE
2,This Criminal Appeal is filed under Section 37...,PREAMBLE
3,This appeal coming on for hearing thi...,PREAMBLE
4,Heard the learned Counsel for the appe...,NONE
...,...,...
28981,So Section 132 of the Evidence Act sufficien...,RATIO
28982,"For the reasons aforesaid, the appeal is all...",RPC
28983,"The judgment and order dated April 27, 1987 pa...",RPC
28984,R.S.S.,NONE


In [6]:
new_ds=ds.loc[ds['Label'].isin(['FAC','ARG_RESPONDENT', 'ARG_PETITIONER', 'ISSUE'])]
#new_ds = ds

In [7]:
unique_classes = new_ds['Label'].unique()
print("Unique classes:", unique_classes)

Unique classes: ['FAC' 'ARG_RESPONDENT' 'ARG_PETITIONER' 'ISSUE']


In [8]:
new_ds[["Label"]] = new_ds[["Label"]].apply(LabelEncoder().fit_transform)

<ipython-input-8-0f393c6c280d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds[["Label"]] = new_ds[["Label"]].apply(LabelEncoder().fit_transform)


In [9]:
new_ds
'''name = "new_ds.csv"
new_ds.to_csv(name, index=False)'''

'name = "new_ds.csv"\nnew_ds.to_csv(name, index=False)'

In [10]:
X = new_ds['Text']
y = new_ds['Label']

In [11]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=47)


In [13]:
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train.to_frame(), y_train)
X_train_resampled = X_train_resampled.squeeze()
y_train_resampled = y_train_resampled.squeeze()

X_test_resampled, y_test_resampled = rus.fit_resample(X_test.to_frame(), y_test)
X_test_resampled = X_test_resampled.squeeze()
y_test_resampled = y_test_resampled.squeeze()


In [14]:

X_train_tensor = tf.convert_to_tensor(X_train_resampled, dtype=tf.string)
y_train_tensor = tf.convert_to_tensor(tf.keras.utils.to_categorical(y_train_resampled, num_classes=13), dtype=tf.float32)

X_test_tensor = tf.convert_to_tensor(X_test_resampled, dtype=tf.string)
y_test_tensor = tf.convert_to_tensor(tf.keras.utils.to_categorical(y_test_resampled, num_classes=13), dtype=tf.float32)

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
outputs['pooled_output']

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer_1')>

In [17]:
X_train_bert_embed = bert_preprocess(X_train_resampled)
X_train_bert_embed = bert_encoder(X_train_bert_embed)["pooled_output"]

X_test_bert_embed = bert_preprocess(X_test_resampled)
X_test_bert_embed = bert_encoder(X_test_bert_embed)["pooled_output"]

In [18]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])

l = tf.keras.layers.Dense(128, activation='gelu', name="hidden")(l)
l = tf.keras.layers.Dropout(0.5, name="dropout_2")(l)

l = tf.keras.layers.Dense(64, activation='gelu', name="hidden_2")(l)
l = tf.keras.layers.Dropout(0.1, name="dropout_3")(l)

l = tf.keras.layers.Dense(32, activation='gelu', name="hidden_3")(l)
l = tf.keras.layers.Dropout(0.1, name="dropout_4")(l)

l = tf.keras.layers.Dense(16, activation='gelu', name="hidden_4")(l)
l = tf.keras.layers.Dropout(0.1, name="dropout_5")(l)

output_layer = tf.keras.layers.Dense(13, activation='softmax', name="output")(l)

In [19]:
model = tf.keras.Model(inputs=[text_input], outputs = [output_layer])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [21]:
METRICS = [tf.keras.metrics.F1Score(
    average=None, threshold=None, name='f1_score', dtype=None
),
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
]
optimizer1 = tf.keras.optimizers.Adafactor(
    learning_rate=0.001,
    beta_2_decay=-0.8,
    epsilon_1=1e-30,
    epsilon_2=0.001,
    clip_threshold=1.0,
    relative_step=True,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adafactor')
optimizer2=tf.keras.optimizers.AdamW(
    learning_rate=0.001,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='AdamW')
optimizer3 = tf.keras.optimizers.Lion(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.99,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Lion')
optimizer4=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adam')

optimizer5=tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.0,
    nesterov=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="SGD")

optimizer6=tf.keras.optimizers.Adadelta(
    learning_rate=0.001,
    rho=0.95,
    epsilon=1e-07,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="Adadelta")

optimizer7=tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="Adagrad")

optimizer8=tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=100,
    jit_compile=True,
    name="RMSprop")


In [22]:
base_models = []

model_1 = tf.keras.models.clone_model(model)
model_1.compile(optimizer=optimizer1, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_1)

model_2 = tf.keras.models.clone_model(model)
model_2.compile(optimizer=optimizer2, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_2)

model_3 = tf.keras.models.clone_model(model)
model_3.compile(optimizer=optimizer3, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_3)

model_4 = tf.keras.models.clone_model(model)
model_4.compile(optimizer=optimizer4, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_4)

model_5 = tf.keras.models.clone_model(model)
model_5.compile(optimizer=optimizer5, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_5)

model_6 = tf.keras.models.clone_model(model)
model_6.compile(optimizer=optimizer6, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_6)

model_7 = tf.keras.models.clone_model(model)
model_7.compile(optimizer=optimizer7, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_7)

model_8 = tf.keras.models.clone_model(model)
model_8.compile(optimizer=optimizer8, loss='categorical_crossentropy', metrics=METRICS)
base_models.append(model_8)

train_predictions = []
for model in base_models:
    train_pred = model.predict(X_train_tensor)
    train_predictions.append(train_pred)


33/33 [==============================] - 12s 346ms/step


In [23]:
stacked_train_predictions = np.hstack(train_predictions)
X_train_combined = np.hstack((X_train_bert_embed, stacked_train_predictions))



In [24]:
test_predictions = []
for model in base_models:
    test_pred = model.predict(X_test_tensor)
    test_predictions.append(test_pred)

stacked_test_predictions = np.hstack(test_predictions)
X_test_combined = np.hstack((X_test_bert_embed, stacked_test_predictions))

14/14 [==============================] - 5s 353ms/step


In [25]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2]
}

In [26]:
xgb_classifier = xgb.XGBClassifier(random_state=42)



In [28]:
grid_search = GridSearchCV(xgb_classifier, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train_combined, np.array(y_train_resampled))

best_n_estimators = grid_search.best_params_['n_estimators']
best_learning_rate = grid_search.best_params_['learning_rate']

In [29]:
print("Best n_estimators:", best_n_estimators)
print("Best learning_rate:", best_learning_rate)

Best n_estimators: 150
Best learning_rate: 0.2


In [30]:
xgb_classifier = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    learning_rate=best_learning_rate,
    n_estimators=best_n_estimators,
    max_depth=3,
    random_state=42
)
model = xgb_classifier.fit(X_train_combined, y_train_resampled)



In [31]:
predicted_labels = model.predict(X_test_combined)
predicted_class_labels = label_encoder.inverse_transform(predicted_labels)




In [32]:
print(predicted_class_labels[0:20])

[3 1 1 1 2 1 1 0 1 2 1 2 1 0 0 2 0 0 1 1]


In [33]:
from sklearn.metrics import confusion_matrix, classification_report

test_predictions = xgb_classifier.predict(X_test_combined)
predicted_class_labels = label_encoder.inverse_transform(test_predictions)
true_class_labels = label_encoder.inverse_transform(y_test_resampled)

conf_matrix = confusion_matrix(true_class_labels, predicted_class_labels)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(true_class_labels, predicted_class_labels)
print("Classification Report:")
print(class_report)


Confusion Matrix:
[[42 42 19  4]
 [33 42 24  8]
 [15 19 66  7]
 [ 5  6  6 90]]
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.39      0.42       107
           1       0.39      0.39      0.39       107
           2       0.57      0.62      0.59       107
           3       0.83      0.84      0.83       107

    accuracy                           0.56       428
   macro avg       0.56      0.56      0.56       428
weighted avg       0.56      0.56      0.56       428



In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [45]:
accuracy = accuracy_score(true_class_labels, predicted_class_labels)
precision = precision_score(true_class_labels, predicted_class_labels, average='macro')
recall = recall_score(true_class_labels, predicted_class_labels, average='macro')
f1 = f1_score(true_class_labels, predicted_class_labels, average='macro')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.5607476635514018
Precision: 0.5567568702370206
Recall: 0.5607476635514018
F1-score: 0.5581646002438082


In [ ]:
##################################################################



In [ ]:
#model.save("NLP_TEXT_CLASSIFICATION_saved_model.h5")


In [49]:
#model = tf.keras.models.load_model("LLP_saved_model")
def check(line):
  line = np.array(line).reshape(1, -1)
  prediction = model.predict(line)
  predicted_labels = np.argmax(prediction, axis=1)
  predicted_class_labels = label_encoder.inverse_transform(predicted_labels)

  dic = {0: 'ANALYSIS', 1: 'ARG_PETITIONER', 2: 'ARG_RESPONDENT', 3: 'FAC', 4: 'ISSUE', 5: 'NONE', 6: 'PREAMBLE', 7: 'PRE_NOT_RELIED', 8: 'PRE_RELIED', 9: 'RATIO', 10: 'RLC', 11: 'RPC', 12: 'STA'}

  for label in predicted_class_labels:
    if label in dic:
      print(dic[label])
#Put your statement here:
line = ['3. The court below answered the above points in the affirmative and accordingly had convicted the accused to undergo simple imprisonment for three years and pay a fine of Rs.5,000/- for the offence punishable under section 498-A IPC and 10 years simple imprisonment and to pay fine of Rs.10,000/- for the offence punishable under Section 306 of the IPC.']
check(line)


ValueError: ignored